# 과제 11. Machine Learning with Earth Engine

In [ ]:
import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize()

### Unsuperviesed classification

#### Create an interactive map

In [ ]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

#### Add data to the map

In [ ]:
# point = ee.Geometry.Point([-122.4439, 37.7538])
point = ee.Geometry.Point([127.78, 36.08])

image = (
    ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
    .filterBounds(point)
    .filterDate("2019-01-01", "2019-12-31")
    .sort("CLOUD_COVER")
    .first()
    .select("B[1-7]")
)

vis_params = {"min": 0, "max": 3000, "bands": ["B5", "B4", "B3"]}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

#### Check image properties

In [ ]:
props = geemap.image_props(image)
props.getInfo()

{'CLOUD_COVER': 0.05,
 'CLOUD_COVER_LAND': 0.05,
 'EARTH_SUN_DISTANCE': 0.986513,
 'ESPA_VERSION': '2_23_0_1b',
 'GEOMETRIC_RMSE_MODEL': 9.295,
 'GEOMETRIC_RMSE_MODEL_X': 5.568,
 'GEOMETRIC_RMSE_MODEL_Y': 7.443,
 'IMAGE_DATE': '2019-11-29',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'LANDSAT_ID': 'LC08_L1TP_115035_20191129_20191216_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1576512850000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0',
 'SATELLITE': 'LANDSAT_8',
 'SENSING_TIME': '2019-11-29T02:05:35.5859070Z',
 'SOLAR_AZIMUTH_ANGLE': 160.754135,
 'SOLAR_ZENITH_ANGLE': 59.869427,
 'SR_APP_VERSION': 'LaSRC_1.3.0',
 'WRS_PATH': 115,
 'WRS_ROW': 35,
 'system:asset_size': '666.019809 MB',
 'system:band_names': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_115035_20191129',
 'system:index': 'LC08_115035_20191129',
 'system:time_end': '2019-11-29 02:05:35',
 'system:time_start': '2019-11-29 02:05:35',
 'system:version': 157692000

In [ ]:
props.get("IMAGE_DATE").getInfo()

'2019-11-29'

In [ ]:
props.get("CLOUD_COVER").getInfo()

0.05

#### Make training dataset

In [ ]:
# Make the training dataset.
training = image.sample(
    **{
        #     'region': region,
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, "training", False)
Map

Map(bottom=26017.0, center=[36.08, 127.78000000000002], controls=(WidgetControl(options=['position', 'transpar…

#### Train the clusterer

In [ ]:
# Instantiate the clusterer and train it.
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

#### Classify the image

In [ ]:
# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "clusters")
Map

Map(bottom=26017.0, center=[36.08, 127.78000000000002], controls=(WidgetControl(options=['position', 'transpar…

#### Label the clusters

In [ ]:
legend_keys = ["One", "Two", "Three", "Four", "etc"]
legend_colors = ["#8DD3C7", "#FFFFB3", "#BEBADA", "#FB8072", "#80B1D3"]

# Reclassify the map
result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(
    result, {"min": 1, "max": 5, "palette": legend_colors}, "Labelled clusters"
)
Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position="bottomright"
)
Map

Map(bottom=51729.0, center=[36.09127994838079, 127.76275634765626], controls=(WidgetControl(options=['position…

#### Visualize the result

In [ ]:
print("Change layer opacity:")
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

#### Add data to the map

In [ ]:
point = ee.Geometry.Point([-122.4439, 37.7538])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
    .filterBounds(point)
    .filterDate("2016-01-01", "2016-12-31")
    .sort("CLOUD_COVER")
    .first()
    .select("B[1-7]")
)

vis_params = {"min": 0, "max": 3000, "bands": ["B5", "B4", "B3"]}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

### Check image properties

In [ ]:
ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo()

'2016-11-18'

In [ ]:
image.get("CLOUD_COVER").getInfo()

0.08

In [ ]:
nlcd = ee.Image("USGS/NLCD/NLCD2016").select("landcover").clip(image.geometry())
Map.addLayer(nlcd, {}, "NLCD")
Map

Map(bottom=25636.0, center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['positio…

In [ ]:
# Make the training dataset.
points = nlcd.sample(
    **{
        "region": image.geometry(),
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, "training", False)

In [ ]:
print(points.size().getInfo())

3583


In [ ]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-122.25798986874739, 38.2706212827936]}, 'id': '0', 'properties': {'landcover': 31}}


#### Train the classifier

In [ ]:
# Use these bands for prediction.
bands = ["B1", "B2", "B3", "B4", "B5", "B6", "B7"]


# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [ ]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 575, 'B2': 814, 'B3': 1312, 'B4': 1638, 'B5': 1980, 'B6': 2091, 'B7': 1967, 'landcover': 31}}


#### Classify the image

In [ ]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "classified")
Map

Map(bottom=25636.0, center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['positio…

#### Render categorical map

In [ ]:
class_values = nlcd.get("landcover_class_values").getInfo()
class_values

[11,
 12,
 21,
 22,
 23,
 24,
 31,
 41,
 42,
 43,
 51,
 52,
 71,
 72,
 73,
 74,
 81,
 82,
 90,
 95]

In [ ]:
class_palette = nlcd.get("landcover_class_palette").getInfo()
class_palette

['476ba1',
 'd1defa',
 'decaca',
 'd99482',
 'ee0000',
 'ab0000',
 'b3aea3',
 '68ab63',
 '1c6330',
 'b5ca8f',
 'a68c30',
 'ccba7d',
 'e3e3c2',
 'caca78',
 '99c247',
 '78ae94',
 'dcd93d',
 'ab7028',
 'bad9eb',
 '70a3ba']

In [ ]:
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

In [ ]:
Map.addLayer(landcover, {}, "Land cover")
Map

Map(bottom=25636.0, center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['positio…

#### Visualize the result

In [ ]:
print("Change layer opacity:")
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

#### Add a legend to the map

In [ ]:
Map.add_legend(builtin_legend="NLCD")
Map

Map(bottom=25636.0, center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['positio…

## Accuracy assessment

In [ ]:
NLCD2016 = ee.Image("USGS/NLCD/NLCD2016").select("landcover")
Map.addLayer(NLCD2016, {}, "NLCD 2016")

In [ ]:
NLCD_metadata = ee.FeatureCollection("users/giswqs/landcover/NLCD2016_metadata")
Map.addLayer(NLCD_metadata, {}, "NLCD Metadata")

In [ ]:
# point = ee.Geometry.Point([-122.4439, 37.7538])  # Sanfrancisco, CA
# point = ee.Geometry.Point([-83.9293, 36.0526])   # Knoxville, TN
point = ee.Geometry.Point([-88.3070, 41.7471])  # Chicago, IL

In [ ]:
metadata = NLCD_metadata.filterBounds(point).first()
region = metadata.geometry()

In [ ]:
metadata.get("2016on_bas").getInfo()

'LC08_2016256'

In [ ]:
doy = metadata.get("2016on_bas").getInfo().replace("LC08_", "")
doy

'2016256'

In [ ]:
ee.Date.parse("YYYYDDD", doy).format("YYYY-MM-dd").getInfo()

'2016-09-12'

In [ ]:
start_date = ee.Date.parse("YYYYDDD", doy)
end_date = start_date.advance(1, "day")

In [ ]:
image = (
    ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
    .filterBounds(point)
    .filterDate(start_date, end_date)
    .first()
    .select("B[1-7]")
    .clip(region)
)

vis_params = {"min": 0, "max": 3000, "bands": ["B5", "B4", "B3"]}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

Map(bottom=25636.0, center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transpa…

In [ ]:
nlcd_raw = NLCD2016.clip(region)
Map.addLayer(nlcd_raw, {}, "NLCD")

### Prepare for consecutive class labels

In [ ]:
raw_class_values = nlcd_raw.get("landcover_class_values").getInfo()
print(raw_class_values)

[11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 51, 52, 71, 72, 73, 74, 81, 82, 90, 95]


In [ ]:
n_classes = len(raw_class_values)
new_class_values = list(range(0, n_classes))
new_class_values

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [ ]:
class_palette = nlcd_raw.get("landcover_class_palette").getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [ ]:
nlcd = nlcd_raw.remap(raw_class_values, new_class_values).select(
    ["remapped"], ["landcover"]
)
nlcd = nlcd.set("landcover_class_values", new_class_values)
nlcd = nlcd.set("landcover_class_palette", class_palette)

In [ ]:
Map.addLayer(nlcd, {}, "NLCD")
Map

Map(bottom=24690.0, center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transpa…

### Make training data

In [ ]:
# Make the training dataset.
points = nlcd.sample(
    **{
        "region": region,
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, "training", False)

In [ ]:
print(points.size().getInfo())

5000


In [ ]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-88.56212124700772, 42.210469414463425]}, 'id': '0', 'properties': {'landcover': 17}}


### Split training and testing

In [ ]:
# Use these bands for prediction.
bands = ["B1", "B2", "B3", "B4", "B5", "B6", "B7"]

# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
sample = image.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

# Adds a column of deterministic pseudorandom numbers.
sample = sample.randomColumn()

split = 0.7

training = sample.filter(ee.Filter.lt("random", split))
validation = sample.filter(ee.Filter.gte("random", split))

In [ ]:
training.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'B1': 174,
  'B2': 223,
  'B3': 614,
  'B4': 368,
  'B5': 4221,
  'B6': 1737,
  'B7': 766,
  'landcover': 17,
  'random': 0.5253550035172192}}

In [ ]:
validation.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '4_0',
 'properties': {'B1': 171,
  'B2': 201,
  'B3': 429,
  'B4': 301,
  'B5': 3478,
  'B6': 1698,
  'B7': 707,
  'landcover': 16,
  'random': 0.756636818947484}}

### Train the classifier

In [ ]:
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)


In [ ]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "classified")
Map

Map(bottom=24690.0, center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transpa…

### Render categorical map

In [ ]:
class_values = nlcd.get("landcover_class_values").getInfo()
print(class_values)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [ ]:
class_palette = nlcd.get("landcover_class_palette").getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [ ]:
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

In [ ]:
Map.addLayer(landcover, {}, "Land cover")
Map

Map(bottom=24690.0, center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transpa…

### Visualize the result

In [ ]:
print("Change layer opacity:")
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [ ]:
Map.add_legend(builtin_legend="NLCD")
Map

Map(bottom=24690.0, center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transpa…

#### Accuracy assessment

In [ ]:
train_accuracy = classifier.confusionMatrix()

In [ ]:
train_accuracy.getInfo()

[[293, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 0, 177, 6, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 2, 14, 0, 0],
 [1, 0, 1, 427, 4, 3, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 17, 0, 0],
 [0, 0, 0, 11, 181, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 2, 7, 94, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 2, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 5, 0, 0, 0, 187, 0, 0, 0, 0, 0, 0, 0, 0, 1, 8, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 34, 0, 0, 0, 0, 3, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
train_accuracy.accuracy().getInfo()

0.9569677970931889

In [ ]:
train_accuracy.kappa().getInfo()

0.9388448268519107

In [ ]:
train_accuracy.producersAccuracy().getInfo()

[[1],
 [0],
 [0.8592233009708737],
 [0.9384615384615385],
 [0.9141414141414141],
 [0.912621359223301],
 [0.8823529411764706],
 [0.9211822660098522],
 [1],
 [0.9285714285714286],
 [0],
 [1],
 [0.8292682926829268],
 [0],
 [0],
 [0],
 [0.8571428571428571],
 [0.9937712344280861],
 [0.8266666666666667],
 [0.9047619047619048]]

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

[[0.979933110367893,
  0,
  0.9672131147540983,
  0.9143468950749465,
  0.9329896907216495,
  0.9215686274509803,
  0.9375,
  0.935,
  0.75,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0.9696969696969697,
  0.9674751929437707,
  0.9841269841269841,
  1]]

In [ ]:
validated = validation.classify(classifier)

In [ ]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '4_0',
 'properties': {'B1': 171,
  'B2': 201,
  'B3': 429,
  'B4': 301,
  'B5': 3478,
  'B6': 1698,
  'B7': 707,
  'classification': 17,
  'landcover': 16,
  'random': 0.756636818947484}}

In [ ]:
test_accuracy = validated.errorMatrix("landcover", "classification")

In [ ]:
test_accuracy.getInfo()

[[123, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 22, 23, 2, 2, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 4, 39, 7, 0],
 [0, 0, 8, 108, 15, 1, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 2, 35, 3, 0],
 [0, 0, 2, 27, 26, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0],
 [0, 0, 1, 2, 16, 21, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 11, 7, 0, 0, 0, 59, 0, 0, 0, 0, 0, 0, 0, 0, 1, 21, 11, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 3, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
test_accuracy.accuracy().getInfo()

0.7069081153588196

In [ ]:
test_accuracy.kappa().getInfo()

0.5713265011404898

In [ ]:
test_accuracy.producersAccuracy().getInfo()

[[0.968503937007874],
 [0],
 [0.19298245614035087],
 [0.6101694915254238],
 [0.37142857142857144],
 [0.4772727272727273],
 [0],
 [0.5363636363636364],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.06521739130434782],
 [0.9162234042553191],
 [0.13636363636363635],
 [0]]

In [ ]:
test_accuracy.consumersAccuracy().getInfo()

[[0.9919354838709677,
  0,
  0.3728813559322034,
  0.5192307692307693,
  0.4,
  0.4883720930232558,
  0,
  0.5175438596491229,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.16666666666666666,
  0.8311218335343787,
  0.11538461538461539,
  0]]

#### Reclassify land cover map

In [ ]:
landcover = landcover.remap(new_class_values, raw_class_values).select(
    ["remapped"], ["classification"]
)

In [ ]:
landcover = landcover.set("classification_class_values", raw_class_values)
landcover = landcover.set("classification_class_palette", class_palette)

In [ ]:
Map.addLayer(landcover, {}, "Final land cover")
Map

Map(bottom=24690.0, center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transpa…